# Project Description and Goals

Insert brief description

### Objectives for this project include:
- Identifying ____.
- Documenting process and analysis throughout the data science pipeline.
- Demonstrating the information that was discovered.
- Deliverables:
    - [README.md](insert link) file containing overall project information, how to reproduce work, and notes from project planning.
    - Jupyter Notebook Report detailing the pipeline process.
    - Python module that automates the data [wrangling](insert link).

### Pipeline Process:
1. Acquire
2. Prepare
3. Explore
4. Model/Evaluate
5. Deliver

### Initial hypotheses
- Does vote count or vote average affect movie revenue?
- Do higher budget films bring more revenue?
- Do more popular movies bring in more revenue?


### Project Imports


# Data Acquisition

### Findings

# Data Preparation

# Data Exploration

# Data Modeling / Evaluation

# Conclusion